In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Dense, multiply, add, Concatenate, Activation, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from PIL import Image, ImageChops, ImageEnhance
import numpy as np
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import pickle
from tensorflow.keras.utils import to_categorical

In [ ]:
def convert_to_ela_image(path, quality, save_path=None):
    original_image = Image.open(path).convert('RGB')
    resaved_file_name = 'resaved_image.jpg'
    original_image.save(resaved_file_name, 'JPEG', quality=quality)
    resaved_image = Image.open(resaved_file_name)
    ela_image = ImageChops.difference(original_image, resaved_image)

    extrema = ela_image.getextrema()
    max_difference = max([pix[1] for pix in extrema])
    if max_difference == 0:
        max_difference = 1
    scale = 255 / max_difference

    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)

    if save_path:
        ela_image.save(save_path)

    return ela_image

def prepare_image(image_path, save_path=None):
    image_size = (128,128)
    ela_image = convert_to_ela_image(image_path, 90, save_path)
    return np.array(ela_image.resize(image_size)) / 255.0

In [ ]:
X = []
Y = []

path_authentic = '/content/drive/MyDrive/Casia-my dataset/Au-ela'
path_tampered = '/content/drive/MyDrive/Casia-my dataset/Tp-ela'
output_path_authentic = '/content/drive/MyDrive/Casia-my dataset/Au-ela'
output_path_tampered = '/content/drive/MyDrive/Casia-my dataset/Tp-ela'

os.makedirs(output_path_authentic, exist_ok=True)
os.makedirs(output_path_tampered, exist_ok=True)

for filename in tqdm(os.listdir(path_authentic), desc="Processing Authentic Images : "):
    full_path = os.path.join(path_authentic, filename)
    save_path = os.path.join(output_path_authentic, filename)
    X.append(prepare_image(full_path,save_path))
    Y.append(1)  # Authentic image label

for filename in tqdm(os.listdir(path_tampered), desc="Processing Tampered Images : "):
    full_path = os.path.join(path_tampered, filename)
    save_path = os.path.join(output_path_tampered, filename)
    X.append(prepare_image(full_path,save_path))
    Y.append(0)  # Tampered image label

X = np.array(X)
#Y= np.array(Y)

Y = to_categorical(Y, 2)

print(f'X shape: {X.shape}')
print(f'Y shape: {Y.shape}')

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=7)
#X = X.reshape(-1,1,1,1)
print(len(X_train), len(y_train))
print(len(X_val), len(y_val))

Attention Block



In [ ]:
class CBAM(Layer):
    def __init__(self, filters, ratio=8):
        super(CBAM, self).__init__()
        self.filters = filters
        self.ratio = ratio

    def build(self, input_shape):
        self.shared_dense_one = Dense(input_shape[-1] // self.ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)
        self.shared_dense_two = Dense(input_shape[-1], kernel_initializer='he_normal', use_bias=False)
        self.conv = Conv2D(1, (7, 7), strides=1, padding='same', activation='sigmoid', kernel_initializer='he_normal', use_bias=False)

    def call(self, input_tensor):
        # Channel Attention
        avg_pool = GlobalAveragePooling2D()(input_tensor)
        max_pool = GlobalMaxPooling2D()(input_tensor)

        # Correct Reshape to keep the number of elements the same
        avg_pool = Reshape((1, 1, avg_pool.shape[-1]))(avg_pool)
        max_pool = Reshape((1, 1, max_pool.shape[-1]))(max_pool)

        avg_out = self.shared_dense_two(self.shared_dense_one(avg_pool))
        max_out = self.shared_dense_two(self.shared_dense_one(max_pool))

        channel_attention = Activation('sigmoid')(add([avg_out, max_out]))
        channel_attention = multiply([input_tensor, channel_attention])

        # Spatial Attention
        avg_pool = tf.reduce_mean(channel_attention, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(channel_attention, axis=-1, keepdims=True)
        spatial_attention = Concatenate(axis=-1)([avg_pool, max_pool])
        spatial_attention = self.conv(spatial_attention)

        spatial_attention = multiply([channel_attention, spatial_attention])
        return spatial_attention

    def compute_output_shape(self, input_shape):
        return input_shape

In [ ]:
def lr_schedule(epoch, lr):
    init_lr = .0007
    total_epochs=70
    decay = init_lr / total_epochs
    new_lr = init_lr * (1 / (1 + decay * (epoch + 30)))

    return max(new_lr, 1e-5)

In [ ]:
def create_model_with_cbam(input_shape=(128,128,3)):
    input_layer = Input(shape=input_shape)

    # Convolutional Block 1
    conv1 = Conv2D(32, (2, 2), activation='relu',padding = 'valid')(input_layer)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    # CNN block 2
    conv2 = Conv2D(64, (2, 2), activation='relu',padding = 'valid')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    #CNN block3
    conv3 = Conv2D(128, (3, 3), activation='relu')(pool2)
    conv4 = Conv2D(128, (3, 3), activation='relu')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv4)

    #CNN block4
    conv5 = Conv2D(256, (3, 3), activation='relu')(pool3)
    conv6 = Conv2D(256, (3, 3), activation='relu',padding='same')(conv5)


    # CBAM Block 2
    cbam1 = CBAM(filters=32)(conv6)


    # Fully Connected Layers
    gap = GlobalAveragePooling2D()(cbam1)

    dense1 = Dense(64, activation='relu')(gap)
    dropout = Dropout(0.4)(dense1)
    dense2 = Dense(128, activation='relu')(dropout)
    dropout = Dropout(0.4)(dense2)
    output_layer = Dense(2, activation='softmax')(dropout)

    model = Model(inputs=input_layer, outputs=output_layer)


    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0007), loss='categorical_crossentropy', metrics=['accuracy'])

    return model


model = create_model_with_cbam()
model.summary()

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler

lr_scheduler = LearningRateScheduler(lr_schedule)
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=500, batch_size=32,callbacks=[early_stopping])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation loss: {loss:.4f}, accuracy: {accuracy:.4f}')

y_pred_prob = model.predict(X_val)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

In [ ]:
Y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors
Y_pred_classes = np.argmax(Y_pred,axis = 1)
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_val,axis = 1)
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(2))